# Quantum Teleportation Protocol - Problem 1 (Part A)
This notebook demonstrates the quantum teleportation algorithm, which enables the transfer of an unknown quantum state between two distant parties using shared entanglement.

### Setup
Loading Qiskit modules for circuit construction, simulation, and result visualization.

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram

### 1. State Initialization
We construct the state to be teleported by rotating qubit 0 with $R_y$ and $R_z$ gates. The rotation parameters $\alpha = 1.2$ and $\gamma = 2.1$ are selected from the allowed interval $[0.5, 2.5]$ to produce a non-trivial superposition state $|\psi\rangle$ on the first qubit.

In [ ]:
alpha = 1.2
gamma = 2.1

qubits = QuantumRegister(3, 'q')
meas_bit = ClassicalRegister(1, 'c')
circ = QuantumCircuit(qubits, meas_bit)

circ.ry(alpha, 0)
circ.rz(gamma, 0)
circ.barrier()

### 2. EPR Pair Generation
Next, an Einstein-Podolsky-Rosen (EPR) pair is formed between qubits 1 and 2. The Hadamard gate places qubit 1 into superposition, and the subsequent CNOT gate entangles it with qubit 2. After this step, Alice holds qubit 1 and Bob holds qubit 2.

In [ ]:
circ.h(1)
circ.cx(1, 2)
circ.barrier()

### 3. Bell Measurement on Alice's Qubits
Alice interacts her state qubit (qubit 0) with her EPR qubit (qubit 1) through a CNOT followed by Hadamard. This maps the joint state of qubits 0 and 1 onto the four Bell states, effectively encoding the teleportation information.

In [ ]:
circ.cx(0, 1)
circ.h(0)
circ.barrier()

### 4. Conditional Recovery Operations
In this fully-quantum version of the protocol, we replace the classical communication step with controlled unitary gates. The controlled-X gate (conditioned on qubit 1) and controlled-Z gate (conditioned on qubit 0) jointly correct Bob's qubit to recover the original state $|\psi\rangle$.

In [ ]:
circ.cx(1, 2)
circ.cz(0, 2)
circ.barrier()

### 5. Undoing the Preparation & Measuring
To verify correctness, we apply the adjoint (inverse) of the state preparation on Bob's qubit. If teleportation succeeded, qubit 2 collapses to $|0\rangle$ deterministically upon measurement. The inverse consists of $R_z(-\gamma)$ then $R_y(-\alpha)$ applied in that order.

In [ ]:
circ.rz(-gamma, 2)
circ.ry(-alpha, 2)
circ.barrier()

circ.measure(2, 0)

circ.draw('mpl')

### 6. Running the Simulation
We run 5000 independent trials using the statevector-based Aer simulator. If every single trial yields the $|0\rangle$ state, the teleportation protocol is validated to be working perfectly.

In [ ]:
simulator = Aer.get_backend('aer_simulator')
execution = simulator.run(circ, shots=5000)
run_result = execution.result()

measured_counts = run_result.get_counts()
print("Measurement counts:", measured_counts)

plot_histogram(measured_counts)